In [0]:
# The purpose of the notebook is to use user doc and post doc approach to generate embedding of public data set 

In [0]:
import pandas as pd
import numpy as np
import re
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models.doc2vec import Doc2Vec
import random
import string
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
import nltk
nltk.download("popular")

In [0]:
from scipy import spatial
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
lemmatizer = WordNetLemmatizer()

In [0]:
def convert2lower(inputstr):
    return inputstr.lower()

def removeNum(inputstr):
    return re.sub(r'\d+','',inputstr)

def removePunc(inputstr):
    for x in inputstr.lower(): 
        if x in punctuations: 
            inputstr = inputstr.replace(x, "")
    return inputstr

def removeWhiteSpace(inputstr):
    return inputstr.strip()

def removeStopWordsandLemmatize(inputstr):
    token = word_tokenize(inputstr)
    result = [i for i in token if not i in stop_words]
    result = [lemmatizer.lemmatize(i) for i in result]
    return ' '.join(result)

def CompletePreprocessingofWord(tweetlist):
    l = []
    for tweet in tweetlist:
        clean_tweet = convert2lower(tweet)
        clean_tweet = removeNum(clean_tweet)
        clean_tweet = removePunc(clean_tweet)
        clean_tweet = removeStopWordsandLemmatize(clean_tweet)
        clean_tweet = removeWhiteSpace(clean_tweet)
        l.append(clean_tweet)
    return l


In [0]:
    def getuniquehashtags(tweetlist):
        allhashtags = []
        for tweet in tweetlist:
            allhashtags.extend(re.findall(r"#(\w+)",str(tweet)))
        return list(set(allhashtags))
    
    def getUniqueMentionedUsers(tweetlist):
        allmentionedUser = []
        for tweet in tweetlist:
            allmentionedUser.extend(re.findall(r"@(\w+)",str(tweet)))
        return list(set(allmentionedUser))
    
    def getAllhashtags(tweetlist):
        allhashtags = []
        for tweet in tweetlist:
            allhashtags.extend(re.findall(r"#(\w+)",str(tweet)))
        return allhashtags
    
    def getAllMentionedUsers(tweetlist):
        allmentionedUser = []
        for tweet in tweetlist:
            allmentionedUser.extend(re.findall(r"@(\w+)",str(tweet)))
        return allmentionedUser
    
    def RemoveTags(tweetlist):
        clean_tweetlist = []
        for tweet in tweetlist:
            tweet = re.sub('@[^\s]+','',str(tweet))
            tweet = re.sub('#[^\s]+','',str(tweet))
            clean_tweetlist.append(tweet)
        return clean_tweetlist  
    
    def RemoveHTTP(tweetlist):
        l = []
        for tweet in tweetlist:    
            clean_tweet = re.match('(.*?)http.*?\s?(.*?)',str(tweet))
            if(clean_tweet):
                l.append(clean_tweet.group(1))
            else:
                l.append(tweet)
            #l.append(re.sub(r"http:\+","",str(tweet)))
        return l
    
    def RemoveHttps(tweetlist):
        l = []
        for tweet in tweetlist:    
            clean_tweet = re.match('(.*?)https.*?\s?(.*?)',str(tweet))
            if(clean_tweet):
                l.append(clean_tweet.group(1))
            else:
                l.append(tweet)
            #l.append(re.sub(r"http:\+","",str(tweet)))
        return l

    def RemoveWWW(tweetlist):
        l = []
        for tweet in tweetlist:    
            clean_tweet = re.match('(.*?)www.*?\s?(.*?)',str(tweet))
            if(clean_tweet):
                l.append(clean_tweet.group(1))
            else:
                l.append(tweet)
            #l.append(re.sub(r"http:\+","",str(tweet)))
        return l
    
    def DeEmojify(tweetlist):
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
        l = []
        for tweet in tweetlist:
            #l.append(emoji_pattern.sub(r'', tweet))
            l.append(tweet.encode('ascii', 'ignore').decode('ascii'))
        return l
    
        
    def getCustomDataFramebylimits(df,lowerlimit):
        main_df = df
        #if(ifUpperLimit):
        #    main_df = main_df[main_df['UserName'].map(main_df['UserName'].value_counts()) < upperlimit]
        main_df = main_df[main_df['UserName'].map(main_df['UserName'].value_counts()) > lowerlimit]
        Corp_df = pd.DataFrame({'author' : main_df['UserName'].unique()})
        Corp_df['alltweets'] = [list(set(main_df['Tweet'].loc[main_df['UserName'] == x['author']])) for _,x in Corp_df.iterrows()]
        Corp_df['Allhashtags'] = [getAllhashtags(x['alltweets']) for _,x in Corp_df.iterrows()]
        Corp_df['Uniquehashtags'] = [getuniquehashtags(x['alltweets']) for _,x in Corp_df.iterrows()]
        Corp_df['UniqueMentionedUsers'] = [getUniqueMentionedUsers(x['alltweets']) for _,x in Corp_df.iterrows()]
        Corp_df['AllMentionedUsers'] = [getAllMentionedUsers(x['alltweets']) for _,x in Corp_df.iterrows()]  
        Corp_df['selectedtweets'] = [DeEmojify(x['alltweets']) for _,x in Corp_df.iterrows()]
        Corp_df['selectedtweets'] = [RemoveHTTP(x['selectedtweets']) for _,x in Corp_df.iterrows()]
        Corp_df['selectedtweets'] = [RemoveHttps(x['selectedtweets']) for _,x in Corp_df.iterrows()]
        Corp_df['selectedtweets'] = [RemoveWWW(x['selectedtweets']) for _,x in Corp_df.iterrows()]
        Corp_df['selectedtweets'] = [CompletePreprocessingofWord(x['selectedtweets']) for _,x in Corp_df.iterrows()]
        #Corp_df['selectedtweets'] = [RemoveTags(x['selectedtweets']) for _,x in Corp_df.iterrows()]   
        #Corp_df['1000StackUserEmbb'] = []
        return Corp_df

In [0]:
def GetPosDocSimilarityMeanfromSimilarEmb(UserEmb):
    SimilarUserCosineList = []
    if(len(UserEmb) == 1):
        return 1.0
    for i in range(len(UserEmb)):
        for j in range(i+1,len(UserEmb)):
            res = 1 - spatial.distance.cosine(UserEmb[i],UserEmb[j])
            SimilarUserCosineList.append(abs(res))
    mean = sum(SimilarUserCosineList) / len(SimilarUserCosineList)
    return mean
def GetPosDocSimilarityMeanfromDifferentEmb(UserEmb1,UserEmb2):
    DifferentUserCosineList = []
    for i in range(len(UserEmb1)):
        for j in range(len(UserEmb2)):
            res = 1 - spatial.distance.cosine(UserEmb1[i],UserEmb2[j])
            DifferentUserCosineList.append(abs(res))
    mean = sum(DifferentUserCosineList) / len(DifferentUserCosineList)
    return mean

In [0]:
Name_list = ['adriande','akshay','BBC','billgates','britneyspears','bruno','cnn','Cr7','drake','espn','fcbarca','harrystyles','iamSrk','instagram','Nasa','nytimes','obama','realMad','salman','sportscenter','srbachan','trump','twitter','virat','youtube']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk import word_tokenize, pos_tag
cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
def GenerateImportantPOStagging(Alltweetlist):
    keywords = []
    for tweet in Alltweetlist:
        token_lis = word_tokenize(tweet)
        tagged_lis = pos_tag(token_lis)
        keyword_l = [word for word, tag in tagged_lis if ((tag=='NNP') or (tag == 'NNPS') or (tag == 'NNS') or (tag == 'NN'))]
        #print(tagged_lis)
        for word in keyword_l[:]:
            if word.startswith('http'):
                keyword_l.remove(word)
        keywords.extend(keyword_l)
    #X=cv.fit_transform(keywords)
    #k = list(cv.vocabulary_.keys())
    return (' '.join(set(keywords)))

In [0]:
#############  Training data for AutoEncoder and gensim models #########################################
Data_F = pd.read_csv("/content/drive/My Drive/Colab Notebooks/training.1600000.processed.noemoticon.csv",encoding = "latin-1")
Data_F = Data_F[Data_F.columns[4:6]]
Data_F.columns = ["UserName","Tweet"]

In [0]:
data_df = getCustomDataFramebylimits(Data_F,20)

In [0]:
data_df.shape

In [0]:
data_df['SignificantPOS_tweets'] = [GenerateImportantPOStagging(x['selectedtweets']) for _,x in data_df.iterrows()]

In [0]:
data_df['SignificantPOS_tweets'][0]

In [0]:
corpus = []
for i in range(4000):
    corpus.append(data_df['SignificantPOS_tweets'][i])

In [0]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i,_d in enumerate(corpus)]

In [0]:
max_epoch = 100
vec_size = 2000
alpha = 0.025
model = Doc2Vec(vector_size = vec_size, alpha = alpha,min_alpha = 0.00025,min_count = 1,dm = 0)
model.build_vocab(tagged_data)

In [0]:
for epoch in range(max_epoch):
    print('iteration{0}'.format(epoch))
    model.train(tagged_data, total_examples = model.corpus_count, epochs = model.epochs)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha
    
    if((epoch % 20) == 0):
        model.save("NewDoc2Vec_"+str(epoch)+"T.model")
        print("Model Saved")
model.save("Final_Doc2Vec_POSincorporate.model")
print("Model Saved")

In [0]:
############### Training and testing Set for AutoEncoder#############

In [0]:
POSDOC2Vecmodel = Doc2Vec.load("/content/NewDoc2Vec_100T.model")
final_Matrix_ds = []
d = word_tokenize(data_df['SignificantPOS_tweets'][0].lower())
vec = POSDOC2Vecmodel.infer_vector(d)
final_Matrix_ds = np.concatenate((final_Matrix_ds,vec), axis = None)
for _,x in data_df.iterrows():
  data = word_tokenize(x['SignificantPOS_tweets'].lower())
  vector = POSDOC2Vecmodel.infer_vector(data)
  final_Matrix_ds = np.vstack((final_Matrix_ds,vector))

In [0]:
import tensorflow as tf
import keras 
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Flatten,Input
from keras.layers import BatchNormalization,Activation
from keras.optimizers import Adam
from keras.losses import mean_squared_error

In [0]:
np.random.shuffle(final_Matrix_ds)
training_data, test_data = final_Matrix_ds[:4500,:],final_Matrix_ds[4500:,:]

In [0]:
encoding_dims = 300
input_dim = Input(shape = (2000,))
encoded = Dense(encoding_dims, activation='relu')(input_dim)
decoded = Dense(2000, activation='tanh')(encoded)
autoencoder = Model(input_dim,decoded)

encoder = Model(input_dim,encoded)

encoded_input = Input(shape=(encoding_dims,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

mse = tf.keras.losses.MeanSquaredError()
autoencoder.compile(optimizer='adadelta', loss=mse)

In [0]:
autoencoder.fit(training_data, training_data,
                epochs=100,
                batch_size=250,
                shuffle=True,
                validation_data=(test_data, test_data))

In [0]:
from google.colab import files
decoder.save('decoder.model')
autoencoder.save('autoencoder.h5')
encoder.save('encoder.h5')


In [0]:
###########################Building embeddings from csv directly############
POSDOC2Vecmodel = Doc2Vec.load("/content/NewDoc2Vec_60T.model")
Encoder_model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/encoder_2000to300.h5')
def ReadandBuildEmbedding(csvname):
  initial_lis = []
  Readfilepath =  "/content/drive/My Drive/Colab Notebooks/tweets/" + str(csvname) + ".csv"
  Data_F = pd.read_csv(Readfilepath,encoding = "latin-1")
  Data_F = Data_F[['Username','Tweet Content']]
  Data_F.columns = ["UserName","Tweet"]
  data_df = getCustomDataFramebylimits(Data_F,20)
  data_df['SignificantPOS_tweets'] = [GenerateImportantPOStagging(x['selectedtweets']) for _,x in data_df.iterrows()]
  for _,x in data_df.iterrows():
    fm_ds = []
    d = word_tokenize(x['SignificantPOS_tweets'].lower())
    vec = POSDOC2Vecmodel.infer_vector(d)
    fm_ds = np.concatenate((fm_ds,vec), axis = None)
    fm_ds = np.vstack((fm_ds,vec))
    #print(fm_ds.shape)
    UserEmbedding = Encoder_model.predict(fm_ds)
    #print(UserEmbedding.shape)
    initial_lis.append(UserEmbedding[0])

  return initial_lis


In [0]:
Main_dic = {}
for name in Name_list:
  Main_dic[name] = ReadandBuildEmbedding(name)

In [0]:
Main_dic['adriande']

In [0]:
labellist = []
tokenlist = []
colorlist = []
#c = 0
def preparedataforTsne(UserEmbedding,label,tokenlist):
  c = len(set(labellist))
  for x in UserEmbedding:
    labellist.append(label)
    tokenlist.append(x)
    colorlist.append(colors[c])
  #c = c + 1 
  return tokenlist

In [0]:
for name in Name_list:
    tokenlist = preparedataforTsne(Main_dic[name],name,tokenlist)

In [0]:
import matplotlib.cm as cm
x = np.arange(10)
ys = [i+x+(i*x)**2 for i in range(25)]
colors = cm.rainbow(np.linspace(0, 1, len(ys)))


In [0]:
tsne_model = TSNE(perplexity = 50,n_components=2,init='pca',n_iter=2500, random_state = 23)
new_values = tsne_model.fit_transform(tokenlist)
x_cord = []
y_cord = []

for value in new_values:
    x_cord.append(value[0])
    y_cord.append(value[1])

In [0]:
plt.figure(figsize=(25, 12)) 
for i in range(len(x_cord)):
    plt.scatter(x_cord[i],y_cord[i], c = colorlist[i])
    plt.annotate(labellist[i],xy=(x_cord[i], y_cord[i]),xytext=(5, 2),textcoords='offset points',ha='right',va='bottom')
plt.show()

In [0]:
User2DocSimilaritySameUser = {}  
for name in Name_list:
    User2DocSimilaritySameUser[name] = GetPosDocSimilarityMeanfromSimilarEmb(Main_dic[name])

In [0]:
User2DocSimilaritySameUser

In [0]:
Main_diff_dic = {}
for main_name in Name_list:
    differentUsersimilarity = {}
    for name in Name_list:
        if(name == main_name):
            pass
        else:
            differentUsersimilarity[name] = GetPosDocSimilarityMeanfromDifferentEmb(Main_dic[main_name],Main_dic[name])
    Main_diff_dic[main_name] = differentUsersimilarity



In [0]:
Main_diff_dic

In [0]:
Difference_inSimilarityScore_PostDoc ={}
for main_name in Name_list:
    SimilarUserScore = User2DocSimilaritySameUser[main_name]
    sum_of_difference = 0
    dic = Main_diff_dic[main_name]
    for name in Name_list:
        if (name == main_name):
            pass
        else:  
            s = abs(SimilarUserScore - dic[name])
            sum_of_difference = sum_of_difference + s
    Difference_inSimilarityScore_PostDoc[main_name] = sum_of_difference / 24

In [0]:
Difference_inSimilarityScore_PostDoc

In [0]:
s = 0
for name in Name_list:
  s = s + User2DocSimilaritySameUser[name]        #0.22, 0.67
s / 25

0.6775363818758464

In [0]:
################## Auto Encoder for MultiView BioBert Combine and Graph Embedding ####################

In [0]:
TrainingMat = np.loadtxt('/content/drive/My Drive/Colab Notebooks/AutoEncoderInputMatrix/TrainingMatNew_Matrix.txt')

In [0]:
import tensorflow as tf
import keras 
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Flatten,Input
from keras.layers import BatchNormalization,Activation
from keras.optimizers import Adam
from keras.losses import mean_squared_error

Using TensorFlow backend.


In [0]:
np.random.shuffle(TrainingMat)
training_data, test_data = TrainingMat[:2500,:],TrainingMat[2500:,:]

In [0]:
encoding_dims = 300
input_dim = Input(shape = (550,))
encoded = Dense(encoding_dims, activation='relu')(input_dim)
decoded = Dense(550, activation='tanh')(encoded)
autoencoder = Model(input_dim,decoded)

encoder = Model(input_dim,encoded)

encoded_input = Input(shape=(encoding_dims,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

mse = tf.keras.losses.MeanSquaredError()
autoencoder.compile(optimizer='adadelta', loss=mse)

In [0]:
autoencoder.fit(training_data, training_data,
                epochs=4000,
                batch_size=250,
                shuffle=True,
                validation_data=(test_data, test_data))

In [0]:
TestDataMatrix = np.loadtxt('/content/drive/My Drive/Colab Notebooks/AutoEncoderInputMatrix/TestingMatNew_Matrix.txt')

In [0]:
(TestDataMatrix[0]).shape

(550,)

In [0]:
ResultMat = np.zeros(300)
for i in range(TestDataMatrix.shape[0]):
  vec = np.zeros(TestDataMatrix.shape[1])
  vec = np.vstack((vec,TestDataMatrix[i]))
  res = encoder.predict(vec)
  ResultMat = np.vstack((ResultMat,res[1]))

In [0]:
ResultMat = ResultMat[1:,:]

In [0]:
fileName = "/content/drive/My Drive/Colab Notebooks/AutoEncoderInputMatrix/MultiViewBioBert_Graph_Emb300_Matrix.txt"
np.savetxt(fileName,ResultMat,fmt='%.8f') 

In [0]:
################## Auto Encoder for MultiView BioBert Full and Graph Embedding and 1786 ####################

In [0]:
TrainingMat = np.loadtxt('/content/drive/My Drive/Colab Notebooks/AutoEncoderInputMatrix/TrainMatrix1786.txt')

In [0]:
import tensorflow as tf
import keras 
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Flatten,Input
from keras.layers import BatchNormalization,Activation
from keras.optimizers import Adam
from keras.losses import mean_squared_error

In [0]:
TrainingMat.shape

(3280, 1786)

In [0]:
np.random.shuffle(TrainingMat)
training_data, test_data = TrainingMat[:2500,:],TrainingMat[2500:,:]

In [0]:
encoding_dims = 300
input_dim = Input(shape = (1786,))
encoded = Dense(encoding_dims, activation='relu')(input_dim)
decoded = Dense(1786, activation='tanh')(encoded)
autoencoder = Model(input_dim,decoded)

encoder = Model(input_dim,encoded)

encoded_input = Input(shape=(encoding_dims,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

mse = tf.keras.losses.MeanSquaredError()
autoencoder.compile(optimizer='adadelta', loss=mse)

In [0]:
autoencoder.fit(training_data, training_data,
                epochs=1000,
                batch_size=250,
                shuffle=True,
                validation_data=(test_data, test_data))

In [0]:
TestDataMatrix = np.loadtxt('/content/drive/My Drive/Colab Notebooks/AutoEncoderInputMatrix/SciBertTestMatrix1786.txt')

In [0]:
ResultMat = np.zeros(300)
for i in range(TestDataMatrix.shape[0]):
  vec = np.zeros(TestDataMatrix.shape[1])
  vec = np.vstack((vec,TestDataMatrix[i]))
  res = encoder.predict(vec)
  ResultMat = np.vstack((ResultMat,res[1]))

In [0]:
ResultMat = ResultMat[1:,:]

In [0]:
ResultMat.shape

In [0]:
fileName = "/content/drive/My Drive/Colab Notebooks/AutoEncoderInputMatrix/Test_SciBERT_1786to300_Matrix.txt"
np.savetxt(fileName,ResultMat,fmt='%.8f') 